In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from tqdm import tqdm
import torch
import random, re

In [ ]:
generator_model_id = "ytu-ce-cosmos/Turkish-Gemma-9b-T1"
topics=[
    "Tarih", "Coğrafya", "Fizik", "Kimya", "Biyoloji", "Edebiyat", "Felsefe", 
    "Teknoloji", "Yapay Zeka", "Psikoloji", "Sosyoloji", "Ekonomi", "Sanat", 
    "Müzik", "Sinema", "Spor", "Astronomi", "Matematik", "Tıp", "Mühendislik",
    "Mitoloji", "Doğa", "Hayvanlar Alemi", "Gastronomi", "Mimari"
]

main_topic = "Genel Kültür"
dataset = []
question_count = 500
BATCH_SIZE = 10
num_batches = question_count // BATCH_SIZE


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(generator_model_id)

generator_model = AutoModelForCausalLM.from_pretrained(
    generator_model_id,
    dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
with tqdm(total=num_batches) as pbar:
    for i in range(num_batches):
        
        current_topic = random.choice(topics)
        
        prompt = (
            f"Bana {current_topic} hakkında {BATCH_SIZE} adet {main_topic} sorusu yaz. "
            f"Yazdığın her soru için 1 adet iyi cevap (doğru ve detaylı) ve 1 adet kötü cevap (yanlış veya saçma) yaz. "
            f"Format tam olarak şu şekilde olsun ve başka bir şey yazma:\n\n"
            f"Q1: Soru cümlesi buraya\n"
            f"A1: İyi cevap buraya\n"
            f"A2: Kötü cevap buraya\n"
            f"Q2: Soru cümlesi buraya...\n"
            f"A1: ...\n"
            f"A2: ...\n"
            f"(Hepsini bu formatta alt alta yaz)"
        )
        
        messages = [
            {"role": "user", "content": prompt}
        ]

        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(generator_model.device)

        terminators = [
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<end_of_turn>")
        ]

        outputs = generator_model.generate(
            input_ids,
            max_new_tokens=4096,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.7 + (i * 0.01),
            top_p=0.9,
            repetition_penalty=1.1
        )

        response = outputs[0][input_ids.shape[-1]:]
        raw_text = tokenizer.decode(response, skip_special_tokens=True).strip()

        raw_text = re.sub(r'<think>.*?</think>', '', raw_text, flags=re.DOTALL).strip()

        pattern = r"Q\d+:\s*(.*?)\s*A1:\s*(.*?)\s*A2:\s*(.*?)(?=Q\d+:|$)"
        matches = re.findall(pattern, raw_text, re.DOTALL)

        for match in matches:
            question, good_ans, bad_ans = match
            
            entry = {
                "Konu": current_topic,
                "Soru": question.strip(),
                "Iyi_Cevap": good_ans.strip(),
                "Kotu_Cevap": bad_ans.strip()
            }
            
            if entry["Soru"] not in [x["Soru"] for x in dataset]:
                dataset.append(entry)
                pbar.update(1)

        if len(dataset) % 20 == 0 and len(dataset) > 0:
            print(f"\nSon eklenen: {dataset[-1]['Soru']}")


In [ ]:
df = pd.DataFrame(dataset)
print(f"\nİşlem tamamlandı! Toplam {len(df)} soru üretildi.")
print(df.head())

df.to_csv("soru_cevap_dataset_500.csv", index=False, encoding='utf-8-sig')